In [9]:
# Gets historical weather data from WunderGround
# Using BeautifulSoup and Selenium to get the monthly weather stats

import requests, sys, re
import pandas as pd

from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.keys import Keys


In [10]:
def get_page(html, attempt=0, threshold=10):
    page = requests.get(html)
    try: 
        assert(199 < page.status_code < 300)
        return page
    except AssertionError:
        if attempt < threshold:
            return get_page(html, attempt + 1, threshold=threshold)
        else:
            raise Exception('Exceeded maximum attempts')

In [12]:
# hopefully converts a US location and time to wunderground url
def location_time_to_url(location, time):
    state, city = location
    year, month = time
    return 'https://www.wunderground.com/history/monthly/us/' + state + '/' + city + '/date/' + year + '-' + month

# parse monthly weather data
def parse_table(table):
    category = 0
    # Hardcoded is bad but idk how to make this dynamic just hope wunderground doesn't change their site layout anytime soon
    headers = ['Time', 'Tempertaure (F)', 'Dew Point (F)', 'Humidity (%)', 'Wind Speed (mph)', 'Pressure (Hg)', 'Precipitation']
    row_data = {'Time': [], 'Tempertaure (F)': [], 'Dew Point (F)': [], 'Humidity (%)': [], 'Wind Speed (mph)': [], 'Pressure (Hg)': [], 'Precipitation': []}
    rows = table.find_all('tr')
    for i in range(2, len(rows)):
        row = rows[i].text.strip()
        if row.lower().islower():
            if i > 2:
                category += 1
        else:
            row_data[headers[category]].append(re.sub('\s+', ',', row))
    return row_data



def fetch_location_data(state, city, year, month):
    url = location_time_to_url((state, city), (year, month))
    bi = FirefoxBinary(r'C:\Program Files (x86)\Mozilla Firefox\\firefox.exe')
    br = webdriver.Firefox(firefox_binary=bi)

    br.get(url)

    soup = bs(br.page_source, 'lxml')
    br.quit()

    table = soup.find_all('table', {'class': 'days ng-star-inserted'})
    parsed_table = parse_table(table[0])
    pd.DataFrame.from_dict(parsed_table).to_csv(state + '_' + city + '_' + year + '_' + month + '.csv', index=False)
    # with open(state + '_' + city + '_' + year + '_' + month + '.csv', 'w') as f:
    #     for key in parsed_table:
    #         f.write("%s,%s\n"%(key, parsed_table[key]))

In [13]:
fetch_location_data('ny', 'manhattan', '2020', '03')